## Here we try to run the simulation with more wiggle space

In [1]:
import sys

import numpy as np
from astropy.time import TimeDelta
import spiceypy as spice

# sys.path.insert(0, "/".join(__file__.split("/")[:-4]))
sys.path.insert(0, "/home/mglos/skola/DIP/LavaTubeSniffer")
from src.SPICE.config import (
    MOON_STR_ID,
    MOON_REF_FRAME_STR_ID,
    TIME_STEP,
    ABBERRATION_CORRECTION,
    LRO_STR_ID,
)
from src.SPICE.instruments import DIVINERInstrument
from src.SPICE.sweep import SweepIterator, Sweeper
from src.SPICE import instruments


In [2]:
sweeper = Sweeper()

Loading lone SPICE kernels from files: 100%|█████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 153.36it/s]
2025-02-21 22:24:50,970 - INFO - Loading detailed model of the Moon


In [ ]:
import cProfile
import pstats

# Define the output profiling file
prof_file = "sweeper_profile.prof"

# Run profiling and save to file
profiler = cProfile.Profile()
profiler.enable()

# Run the simulation with profilingstepstep
sweeper.run_simulation(2000)

profiler.disable()
profiler.dump_stats(prof_file)  # Save results

print(f"✅ Profiling complete. Results saved to {prof_file}")


Simulation step:0/2000; failed: 0; found: 0:   0%|                                                       | 1.024/479545778.789 [00:00<92:36:35, 1438.37it/s]

> /home/mglos/skola/DIP/LavaTubeSniffer/src/SPICE/sweep.py(151)run_simulation()
    149                         if not feasible_points:
    150                             import pdb; pdb.set_trace()
--> 151                         new_distances = self.compute_distances_from_projected_vector(feasible_points, boresight)
    152                         boresights.append(boresight)
    153                         if any(new_distances < instrument.rough_treshold):



ipdb>  self.compute_distances_from_projected_vector(feasible_points, boresight)


array([], dtype=float64)


ipdb>  feasable_points


...

ipdb>  feasible_points - boresight


array([], shape=(0, 3), dtype=float64)


ipdb>  c


Simulation step:0/2000; failed: 0; found: 0:   0%|                                        | 428.00707537268676/479545778.789 [11:34<216050:00:18,  1.62s/it]

> /home/mglos/skola/DIP/LavaTubeSniffer/src/SPICE/sweep.py(150)run_simulation()
    148                         feasible_points, feasible_ids, boresight = self.find_nearby_pits(instrument)
    149                         if not feasible_points:
--> 150                             import pdb; pdb.set_trace()
    151                         new_distances = self.compute_distances_from_projected_vector(feasible_points, boresight)
    152                         boresights.append(boresight)



In [17]:
#sweeper.kd_tree.query_ball_point(np.array([1700,0,0]), 500)

x = np.ones((1000,3))
x - np.array([])

ValueError: operands could not be broadcast together with shapes (1000,3) (0,) 

In [4]:
import pickle
import datetime

filename = '/home/mglos/skola/' + datetime.datetime.now().isoformat() + "simulation_DIVINER_data.pkl"
with open(filename, "wb") as f:
    pickle.dump(sweeper.data,f)

In [7]:
import pickle
filename = "/home/mglos/skola/2025-02-20T02:23:17.885779simulation_DIVINER_data.pkl"
with open(filename, "rb") as f:
    data = pickle.load(f)

In [21]:
data_found = np.array([sweep[0] for sweep in data['found']])
data_boresight = np.array([sweep[0] for sweep in data['boresight_projections']])

In [37]:
#x = data_found[::100, :]
y = data_boresight[::400, :]
x = data_found[(np.linalg.norm((data_found - np.array([1136.98541918,  477.90804725, 1221.38220844])), axis=1) < 20)]

In [38]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import plotly
from scipy.spatial import cKDTree
from tqdm import tqdm

# Ensure Plotly is set up for Jupyter Notebook
pio.renderers.default = "notebook"

def plot_moon_model(points, highlight_points=None):
    """
    Plots a 3D scatter plot of moon surface points with color-mapped elevation.
    If a second dataset (highlight_points) is provided, it is plotted with colors based on distance to the nearest point in 'points'.

    Parameters:
    - points (array-like): A list or NumPy array of shape (n, 3) representing x, y, z coordinates.
    - highlight_points (array-like, optional): Another array of shape (m, 3) to be plotted with color mapping based on distance.

    Returns:
    - None - Displays the interactive plot in Jupyter Notebook.
    """
    if isinstance(points, list):
        points = np.array([np.asarray(p).flatten() for p in points])

    if points.ndim != 2 or points.shape[1] != 3:
        raise ValueError(f"Expected (n, 3) shape but got {points.shape}")

    # Compute radial distances and elevation (deviation from mean radius)
    radii = np.linalg.norm(points, axis=1)
    avg_radius = np.mean(radii)
    elevations = radii - avg_radius

    # Main dataset scatter plot (colored by elevation)
    scatter = go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=elevations,  # No normalization needed
            colorscale='Viridis',  # Old one for elevationz
            colorbar=dict(title='Elevation', x=0.9),  # Separate legend position
        ),
        name="Main Data (Elevation)"
    )

    data = [scatter]  # Store the main plot

    # If highlight_points exist, compute distances to the closest main point
    if highlight_points is not None:
        if isinstance(highlight_points, list):
            highlight_points = np.array([np.asarray(p).flatten() for p in highlight_points])

        if highlight_points.ndim != 2 or highlight_points.shape[1] != 3:
            raise ValueError(f"Expected (m, 3) shape but got {highlight_points.shape}")

        # Use KD-tree for fast nearest neighbor search
        tree = cKDTree(points)
        distances, _ = tree.query(highlight_points)  # Get nearest neighbor distances

        # Avoid zero distances for log scaling
        distances_scaled = np.log10(distances + 1e-6)  # Apply log scale to colors only

        highlight_scatter = go.Scatter3d(
            x=highlight_points[:, 0],
            y=highlight_points[:, 1],
            z=highlight_points[:, 2],
            mode='markers',
            marker=dict(
                size=7,  # Slightly larger
                color=distances_scaled,  # Log-scale the color but show normal distances
                colorscale='Turbo',  # Better color contrast and peaks
                colorbar=dict(
                    title='Distance to Closest Point',
                    tickvals=np.log10([1e-6, 1e-3, 1e-2, 1e-1, 1, 10, 100]),  # Logarithmic scale ticks
                    ticktext=[r"$10^{-6}$", r"$10^{-3}$", r"$10^{-2}$", r"$10^{-1}$", "1", "10", "100"],  # Normal numbers
                    x=1.05,  # Separate legend position
                ),
                opacity=0.85
            ),
            name="Highlighted Points (Distance)"
        )
        data.append(highlight_scatter)

    # Set equal axis limits
    max_val = np.max(np.abs(points)) * 1.1
    axis_range = [-max_val, max_val]

    layout = go.Layout(
        scene=dict(
            xaxis=dict(range=axis_range, title="X"),
            yaxis=dict(range=axis_range, title="Y"),
            zaxis=dict(range=axis_range, title="Z"),
            aspectmode='cube'
        ),
        title="Moon Surface Model with Log-Scaled Highlights"
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()

# Example usage
plotly.offline.init_notebook_mode(connected=True)

# Convert sweeper.boresights to NumPy array and filter points
#bbb = np.array(sweeper.boresights).reshape((-1, 3))
#filtered_points = bbb[(bbb[:, 0] > 0) & (bbb[:, 1] > 0) & (bbb[:, 2] > 0)]
#filtered_points = bbb

# Create a second set of points to highlight
#highlighted_points = np.array(sweeper._target_points).reshape((-1,3))

#pts = []
#for h in tqdm(highlighted_points):
    #h = h.reshape((1, 3))
    #if ((np.linalg.norm(filtered_points - h, axis=1)).min() < 1000).any():
        #pts.append(h)

# Plot with both datasets
#plot_moon_model(filtered_points, highlight_points=highlighted_points)
#plot_moon_model(filtered_points, highlight_points=pts)
plot_moon_model(x, highlight_points=None)


In [12]:
sweeper._step_time()

IndexError: list index out of range

In [6]:
len(sweeper.boresights)

0

In [5]:
len(sweeper._failed_timestamps)

9604

In [7]:
xxx = []
pts = []
from tqdm import tqdm
for h in tqdm(highlighted_points):
    h = h.reshape((1, 3))
    if ((np.linalg.norm(filtered_points - h, axis=1)).min() < 1000).any():
        pts.append(h)
    #xxx.append(np.linalg.norm(filtered_points - h, axis=1))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 278/278 [00:00<00:00, 1140.32it/s]


In [54]:
b = np.array(sweeper.boresights).reshape((-1, 1, 3))
b.shape
p = np.array(pts)

In [55]:
p = p.reshape((1, -1, 3))
d = np.linalg.norm((b - p).reshape((-1, 3)), axis=1)

In [56]:
b.shape, p.shape, d.shape

((70820, 1, 3), (1, 217, 3), (15367940,))

In [58]:
dd = np.sort(d)
dd[:100]

array([ 2.40825872,  3.06728077,  3.54037572,  4.69051272,  5.4841231 ,
        5.5623859 ,  5.69801118,  5.70208408,  6.2245371 ,  6.22726199,
        6.2291961 ,  6.38124078,  6.47132722,  6.63535458,  6.76668959,
        6.79709278,  6.80261231,  6.88101376,  7.07131594,  7.31929578,
        7.38130983,  7.39137981,  7.43689497,  7.44133103,  7.44319605,
        7.57508075,  7.58302937,  7.6289445 ,  7.69277322,  7.69341574,
        7.7125289 ,  7.90132585,  8.06548866,  8.15521697,  8.16697768,
        8.22000298,  8.34486046,  8.39951556,  8.41105717,  8.45271401,
        8.47466354,  8.5868484 ,  8.66005828,  8.71895071,  8.91546381,
        8.9459852 ,  8.95316047,  8.96344306,  8.98379215,  8.99829232,
        9.0026174 ,  9.06006262,  9.0616007 ,  9.18376841,  9.22938089,
        9.24892812,  9.3034    ,  9.42202353,  9.48418003,  9.57582896,
        9.66819889,  9.78470069, 10.04702739, 10.05446819, 10.16326534,
       10.16483771, 10.22252237, 10.22639227, 10.24805545, 10.29

In [4]:
import plotly.io as pio
import plotly.graph_objects as go
import pandas as pd

pio.renderers.default = "vscode"

def plot_sweeper_points(df):
    """
    Plots a 3D scatter plot of X, Y, Z coordinates from the provided DataFrame.
    """
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=df["X"], y=df["Y"], z=df["Z"],
        mode='markers',
        marker=dict(
            size=3,
            color=color,  # Coloring based on Z values
            colorscale='Viridis',
            opacity=0.8
        ),
        name="Sweeper Points"
    ))

    # Set aspect ratio based on data range
    x_diff = df["X"].max() - df["X"].min()
    y_diff = df["Y"].max() - df["Y"].min()
    z_diff = df["Z"].max() - df["Z"].min()
    aspect_ratio = dict(x=1, y=y_diff / x_diff, z=z_diff / x_diff)

    fig.update_layout(
        title="3D Points Visualization - Sweeper Data",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            aspectmode="manual",
            aspectratio=aspect_ratio
        ),
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()

# Assuming sweeper.points is your DataFrame
#plot_sweeper_points(sweeper._points)
dd = np.array(b)
ddd = {
    'X': dd[:, 0],
    'Y': dd[:, 1],
    'Z': dd[:, 2]
}

color = np.linalg.norm(dd, axis=1)

plot_sweeper_points(pd.DataFrame(ddd))



NameError: name 'b' is not defined

In [4]:
b = [bbb[0] for bbb in sweeper.boresights]

In [10]:
import plotly.io as pio
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

pio.renderers.default = "vscode"

def cluster_sweeper_points(df, eps=500, min_samples=5):
    """
    Applies DBSCAN clustering to X, Y, Z points and returns the DataFrame with cluster labels.
    """
    xyz = df[["X", "Y", "Z"]].values
    xyz_scaled = StandardScaler().fit_transform(xyz)  # Normalize data for better clustering

    db = DBSCAN(eps=eps, min_samples=min_samples).fit(xyz_scaled)
    df["Cluster"] = db.labels_  # Assign cluster labels (-1 means noise)
    
    return df

def plot_clustered_sweeper_points(df):
    """
    Plots a 3D scatter plot of clustered X, Y, Z points using Plotly.
    """
    fig = go.Figure()

    unique_clusters = sorted(df["Cluster"].unique())  # Get unique cluster IDs

    for cluster in unique_clusters:
        cluster_data = df[df["Cluster"] == cluster]
        color = "black" if cluster == -1 else None  # Black for noise points

        fig.add_trace(go.Scatter3d(
            x=cluster_data["X"], y=cluster_data["Y"], z=cluster_data["Z"],
            mode='markers',
            marker=dict(
                size=3,
                color=cluster_data["Cluster"],  # Color by cluster
                colorscale="Viridis" if color is None else None,
                opacity=0.8
            ),
            name=f"Cluster {cluster}" if cluster != -1 else "Noise"
        ))

    # Set aspect ratio
    x_diff = df["X"].max() - df["X"].min()
    y_diff = df["Y"].max() - df["Y"].min()
    z_diff = df["Z"].max() - df["Z"].min()
    aspect_ratio = dict(x=1, y=y_diff / x_diff, z=z_diff / x_diff)

    fig.update_layout(
        title="3D Clustered Points Visualization",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            aspectmode="manual",
            aspectratio=aspect_ratio
        ),
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()

# Apply clustering and plot
sweeper.points = cluster_sweeper_points(sweeper._points)
plot_clustered_sweeper_points(sweeper.points)


In [5]:
for i in range(1):
    try:
        #sweeper._step_time()
        fovs = sweeper._compute_instrument_rays()
        distances = sweeper._compute_fov_distances(fovs, None)
    except Exception as e:
        print(e, sweeper.current_simulation_step)

_compute_fov_distances() takes 2 positional arguments but 3 were given 100


In [5]:
len(sweeper._failed_timestamps), d, len(sweeper._failed_timestamps)

(8505, 85, 8505)

In [12]:
et = sweeper.current_simulation_timestamp_et
bnds = sweeper.instruments[0].compute_views_subinstruments_bounds(et)

brs = sweeper.instruments[0].compute_views_instrument_boresight(et)
_brs = brs['boresight']

rv = []
for _id, val in bnds.items():
    rv.append(np.linalg.norm(val['bounds'] - _brs, axis=1))

# bnds

In [13]:
rv

[array([2.26660674, 2.45364896, 2.4746574 , 2.2849852 ]),
 array([1.9409709 , 1.98882013, 1.97074455, 1.92273225]),
 array([1.72723497, 1.75322272, 1.70663236, 1.67577538]),
 array([1.81557503, 1.75305178, 1.68222367, 1.73678433]),
 array([2.12917745, 2.03064162, 1.90881517, 2.00452994]),
 array([2.49254261, 2.37836653, 2.25085864, 2.36157414]),
 array([2.46265583, 2.32251244, 2.31170245, 2.46662999]),
 array([1.72405723, 1.70147224, 1.67202814, 1.69150882]),
 array([2.29635833, 2.50595027, 2.50059233, 2.27370873])]

In [68]:
kernel_pool

['/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009170_2009172_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009172_2009182_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009182_2009192_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009192_2009202_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009202_2009213_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009213_2009223_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009223_2009233_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009233_2009244_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009244_2009254_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009254_2009264_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009264_2009274_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009274_2009284_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009284_2009294_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009294_2009305_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2009305_2009315_v01.bc',
 '/media/mglos/HDD_8TB1/SPICE/ck/lrosc_2

In [4]:
"""
This script processes SPICE data to identify time intervals when LRO DIVINER
is within areas of interest.
"""

import logging
import sys
from typing import Optional, Generator

sys.path.insert(0, "/".join(__file__.split("/")[:-4]))

import numpy as np
from astropy.time import Time, TimeDelta
import spiceypy as spice
from tqdm import tqdm
from scipy.spatial import cKDTree

from src.global_config import TQDM_NCOLS
from src.scripts.SPICE.instruments import DIVINERInstrument
from src.scripts.SPICE.config import TIME_STEP, MOON_STR_ID, LRO_SPEED, MAX_TIME_STEP
from src.db.mongo.interface import Sessions
from src.scripts.SPICE.sweep_iterator import SweepIterator

logger = logging.getLogger(__name__)


class Sweeper:
    """
    Iterates through SPICE data to update instrument distances to areas of interest.
    """

    def __init__(self):
        self.sweep_iterator = SweepIterator()
        self.computation_timedelta = TimeDelta(TIME_STEP, format="sec")
        self.current_simulation_timestamp = self.sweep_iterator.min_loaded_time + self.computation_timedelta
        self.current_simulation_timestamp_et = spice.str2et(self.current_simulation_timestamp.utc.iso)
        self.current_simulation_step = 0

        # Load instruments
        self.instruments = [DIVINERInstrument()]

        # Get points of interest and build a KD-Tree for fast spatial searches
        self._points = self._get_crater_points_df()
        self._target_points = self._points[["X", "Y", "Z"]].values
        self._target_ids = np.arange(len(self._target_points))
        self.kd_tree = cKDTree(self._target_points)

        # Set simulation start time considering instrument offset
        offset = max(instrument.offset_days for instrument in self.instruments)
        self._set_time(self.sweep_iterator.min_loaded_time + TimeDelta(offset, format="jd"))
        self.sweep_iterator.initiate_sweep(self.current_simulation_timestamp)

        self._failed_timestamps = []
        self._failed_timestamps_kernel_loads = []
        self.adjusted_timesteps = []
        self.min_distances = []

    @property
    def min_time(self):
        return self.sweep_iterator.min_loaded_time

    @property
    def max_time(self):
        return self.sweep_iterator.max_loaded_time

    def _get_crater_points_df(self):
        points = Sessions.get_all_pits_points()
        lat_rad = points["latitude"] * spice.rpd()
        lon_rad = points["longitude"] * spice.rpd()

        # Convert lat/lon to Cartesian coordinates
        cartesian_points = lat_rad.combine(
            lon_rad,
            lambda lat, lon: spice.srfrec(spice.bodn2c(MOON_STR_ID), lon, lat),
        )
        points["X"] = cartesian_points.apply(lambda xyz: xyz[0])
        points["Y"] = cartesian_points.apply(lambda xyz: xyz[1])
        points["Z"] = cartesian_points.apply(lambda xyz: xyz[2])
        return points

    def _step_time(self):
        self.current_simulation_timestamp += self.computation_timedelta
        self.current_simulation_timestamp_et = spice.str2et(self.current_simulation_timestamp.utc.iso)
        self.current_simulation_step += 1

    def _set_time(self, time: Time, timestep: Optional[int] = None):
        self.current_simulation_timestamp = time
        self.current_simulation_timestamp_et = spice.str2et(time.utc.iso)
        self.current_simulation_step = 0 if timestep is None else timestep
        self.sweep_iterator.step(self.current_simulation_timestamp)

    def compute_instrument_rough_distances(self, points, boresight):
        """
        Compute distances from the instrument's boresight to given points.
        """
        return np.linalg.norm(points - boresight, axis=1)

    def find_nearby_pits(self, instrument):
        """
        Use the KD-Tree to find pits within the instrument's rough threshold.
        """
        intersection = instrument.compute_views_instrument_boresight(self.current_simulation_timestamp_et)
        boresight = intersection["boresight"]
        nearby_indices = self.kd_tree.query_ball_point(boresight, instrument.rough_treshold)
        return self._target_points[nearby_indices], self._target_ids[nearby_indices], boresight

    def adjust_timestep(self, boresights):
        min_distance = min(self.kd_tree.query(boresight)[0] for boresight in boresights)
        new_time_step = min_distance / LRO_SPEED
        self.computation_timedelta = TimeDelta(min(max(TIME_STEP, new_time_step), MAX_TIME_STEP), format="sec")
        self.adjusted_timesteps.append(new_time_step)
        self.min_distances.append(min_distance)

    def run_simulation(self, max_steps: Optional[int] = None):
        self.found = 0
        self.boresights = []
        total_seconds = (self.max_time - self.min_time).to_value("sec")
        total_seconds = min(total_seconds, max_steps * TIME_STEP) if max_steps is not None else total_seconds
        with tqdm(total=total_seconds, ncols=TQDM_NCOLS, desc="Running simulation") as pbar:
            while self.current_simulation_timestamp <= self.max_time:
                if max_steps is not None and self.current_simulation_step >= max_steps:
                    break
                try:
                    pbar.update(self.computation_timedelta.to_value("sec"))
                    self._step_time()

                    boresights = []
                    for instrument in self.instruments:
                        feasible_points, feasible_ids, boresight = self.find_nearby_pits(instrument)
                        new_distances = self.compute_instrument_rough_distances(
                            feasible_points, boresight
                        )
                        boresights.append(boresight)
                        if any(new_distances < instrument.rough_treshold):
                            self.found += 1
                            # logging.info("Found!")
                    self.adjust_timestep(boresights)
                    self.boresights.append(boresights)
                except Exception as e:
                    # logger.exception(
                    #     f"Error at step {self.current_simulation_step}: {e}"
                    # )
                    self._failed_timestamps.append((self.current_simulation_timestamp, self.current_simulation_step))
        return


2025-02-15 21:12:51,513 - DEBUG - Loading SPICE for ck/lrodv at 2012-03-31T16:50:24.211
2025-02-15 21:12:51,519 - DEBUG - Loading SPICE /media/mglos/HDD_8TB1/SPICE/ck/lrodv_2012060_2012092_v01.bc
2025-02-15 21:12:51,520 - DEBUG - Loaded SPICE /media/mglos/HDD_8TB1/SPICE/ck/lrodv_2012060_2012092_v01.bc
2025-02-15 21:12:51,521 - DEBUG - Loading SPICE for ck/lrosc at 2012-03-31T16:50:24.211
2025-02-15 21:12:51,537 - DEBUG - Loading SPICE /media/mglos/HDD_8TB1/SPICE/ck/lrosc_2012081_2012092_v02.bc
2025-02-15 21:12:51,539 - DEBUG - Loaded SPICE /media/mglos/HDD_8TB1/SPICE/ck/lrosc_2012081_2012092_v02.bc
2025-02-15 21:12:51,540 - DEBUG - Loading SPICE for spk at 2012-03-31T16:50:24.211
2025-02-15 21:12:51,543 - DEBUG - Loading SPICE /media/mglos/HDD_8TB1/SPICE/spk/lrorg_2012076_2012169_v01.bsp
2025-02-15 21:12:51,544 - DEBUG - Loaded SPICE /media/mglos/HDD_8TB1/SPICE/spk/lrorg_2012076_2012169_v01.bsp


{-85211: Instrument.ProjectedPoint(instrument_id=-85211, et=386484690.3966551, boresight=array([-299.72139795, -419.57003696, 1659.77971413]), boresight_trgepc=386484690.39600927, bounds=[array([-294.06564914, -423.44156778, 1659.84422221]), array([-293.54357447, -422.52823518, 1660.12124121]), array([-305.27009298, -415.55921665, 1659.54631341]), array([-305.78402014, -416.47039994, 1659.24913719])], bounds_trgepc=[386484690.39600897, 386484690.3960087, 386484690.396008, 386484690.39600825]),
 -85212: Instrument.ProjectedPoint(instrument_id=-85212, et=386484690.3966551, boresight=array([-300.97382214, -421.78046011, 1659.03767198]), boresight_trgepc=386484690.3960097, bounds=[array([-295.13062621, -425.36273152, 1659.16858305]), array([-294.94811421, -425.04090734, 1659.28269311]), array([-306.66802606, -418.14185421, 1658.4841631 ]), array([-306.84122552, -418.44829642, 1658.31399036])], bounds_trgepc=[386484690.3960092, 386484690.3960092, 386484690.3960079, 386484690.3960077]),
 -85

In [4]:
from influxdb_client import InfluxDBClient, Point, WritePrecision

class InfluxInterface:
    def __init__(self, url: str, token: str, org: str, bucket: str):
        self.client = InfluxDBClient(url=url, token=token, org=org)
        self.write_api = self.client.write_api()
        self.org = org
        self.bucket = bucket

    def write_simulation_result(self, instrument: str, astro_timestamp, simulation_step: int, boresight, min_distance: float):
        """
        Writes a simulation hit to InfluxDB.
        
        Parameters:
          - instrument: Instrument name (used as a tag).
          - astro_timestamp: An astropy Time object (converted to ISO string).
          - simulation_step: Simulation step number.
          - boresight: A 3D vector (x, y, z) representing the hit location.
          - min_distance: The computed minimum distance.
        """
        # Use the provided astro timestamp in ISO format
        timestamp_iso = astro_timestamp.utc.iso
        point = (
            Point("simulation_results")
            .tag("instrument", instrument)
            .field("min_distance", float(min_distance))
            .field("x", float(boresight[0]))
            .field("y", float(boresight[1]))
            .field("z", float(boresight[2]))
            .field("simulation_step", simulation_step)
            .time(timestamp_iso, WritePrecision.NS)
        )
        self.write_api.write(bucket=self.bucket, org=self.org, record=point)

    def close(self):
        self.client.close()


In [14]:
influx_db = InfluxInterface(
    url="http://localhost:8086",
    token="supersecrettoken",
    org="sat_tracking",
    bucket="telemetry"
)

url="http://localhost:8086"
token="supersecrettoken"
org="sat_tracking"
bucket="telemetry"

In [15]:
import random
from astropy.time import Time

# Generate 10 random simulation results
for i in range(10):
    instrument = f"Instrument-{i%3}"  # Simulating 3 different instruments
    astro_timestamp = Time.now()  # Current timestamp
    simulation_step = i
    boresight = (
        random.uniform(-100, 100),  # x
        random.uniform(-100, 100),  # y
        random.uniform(-100, 100)   # z
    )
    min_distance = random.uniform(0, 50)  # Random distance

    # Write the data to InfluxDB
    influx_db.write_simulation_result(instrument, astro_timestamp, simulation_step, boresight, min_distance)

print("Simulation data written to InfluxDB.")

Simulation data written to InfluxDB.


In [22]:
import pandas as pd
query_api = influx_db.client.query_api()
# Query data from the last 1 hour
query = f"""
from(bucket: "{bucket}")
  |> range(start: -1h)
  |> filter(fn: (r) => r._measurement == "simulation_results")
"""

# Execute query
tables = query_api.query(query)

# Convert to DataFrame for better readability
data = []
for table in tables:
    for record in table.records:
        data.append(record.values)

df = pd.DataFrame(data)

df

,result,table,_start,_stop,_time,_value,_field,_measurement,instrument
0,_result,0,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:08:54.813000+00:00,20.836144,min_distance,simulation_results,Instrument-0
1,_result,0,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:08:54.816000+00:00,14.781655,min_distance,simulation_results,Instrument-0
2,_result,0,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:08:54.818000+00:00,48.536656,min_distance,simulation_results,Instrument-0
3,_result,0,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:08:54.819000+00:00,0.919719,min_distance,simulation_results,Instrument-0
4,_result,0,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:13:30.520000+00:00,45.504203,min_distance,simulation_results,Instrument-0
...,...,...,...,...,...,...,...,...,...
195,_result,14,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:13:32.716000+00:00,73.395161,z,simulation_results,Instrument-2
196,_result,14,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:13:32.722000+00:00,88.884587,z,simulation_results,Instrument-2
197,_result,14,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:15:53.191000+00:00,33.539454,z,simulation_results,Instrument-2
198,_result,14,2025-02-21 21:29:31.243259+00:00,2025-02-21 22:29:31.243259+00:00,2025-02-21 22:15:53.194000+00:00,-76.714593,z,simulation_results,Instrument-2


In [20]:
query = f"""
from(bucket: "{bucket}")
  |> range(start: -24h)
  |> filter(fn: (r) => r._measurement == "simulation_results")
  |> filter(fn: (r) => r.min_distance <= 200)
"""

tables = query_api.query(query)

data = [record.values for table in tables for record in table.records]
df = pd.DataFrame(data)
df

""
